### Solutions

#### Question 1

Write a decorator that can be used to print out how long a function takes to run.

In [1]:
from time import perf_counter

In [3]:
def log(func):
    def inner(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        end = perf_counter()
        print('elapsed time:', end - start)
        return result
    return inner

def add(a, b):
    return a + b
def sub(a, b):
    return a - b
def div(a, b):
    return a / b
def mult(a, b):
    return a * b

@log
def calc(a, b, *, func):
    return func(a, b)

@log
def greet(name):
    return "Hello, {0}".format(name)

In [4]:
calc(1, 2, func=add)

elapsed time: 3.9000005926936865e-06


3

In [5]:
greet('Allwell')

elapsed time: 4.1000021155923605e-06


'Hello, Allwell'

#### Question 2

We have several functions in our code that perform some calculations and return a numeric result, possibly `float`, `int` or even `Decimal`.

We actually want to make sure that all results from each of these functions are rounded to some number of digits after the decimal point (precision), and always returned as a `float`.

But every time our program runs, that precision could change. Also, we'd rather not have to change every function we have, since at some point in the future we may want to return `Decimal` objects, and not `floats` - so we want to minimize how much code we would have to change to accomodate all this.

For example, we might a variable in our code that defines the precision, and could be changed any time we run our code:

In [28]:
PRECISION = 2

Suppose we have the following functions already defined:

In [6]:
from decimal import Decimal

def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0
    
def sum_squares(*args):
    return sum(x**2 for x in args)

def avg(*args):
    if len(args) == 0:
        return 0
    
    numbers = [Decimal(x) for x in args]
    sum_ = sum(numbers)
    return sum_ / len(numbers)

Write a decorator named `normalize` that can be used to decorate these functions to ensure that the result is always returned as a `float` with a precision defined by some global variable `PRECISION`.

In [8]:
def normalize(func):
    def inner(*args, precision):
        result = func(*args)
        return round(result, precision)
    return inner

In [9]:
@normalize
def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0

@normalize
def sum_squares(*args):
    return sum(x**2 for x in args)

@normalize
def avg(*args):
    if len(args) == 0:
        return 0
    
    numbers = [Decimal(x) for x in args]
    sum_ = sum(numbers)
    return sum_ / len(numbers)

In [29]:
perc_diff(3, 4, precision=PRECISION)

33.33

In [30]:
sum_squares(2, 5, 3, 7, 9, 10.2355, precision=PRECISION)

272.77

In [31]:
avg(2.8, 2.1, 9.0, 10, 0.6, 0.2353, precision=PRECISION)

Decimal('4.12')

#### Question 3

Sometimes we have functions that get called often with the same argument values but take a long time to run.

If those functions are deterministic (i.e. passing the same arguments will always result in the same return value), then we can get a huge performance benefit by implementing a caching mechanism.

This function simulates a long running function:

In [32]:
from time import sleep

def add(x, y):
    sleep(2)
    return x + y

As you can see the function is deterministic - the result will always be the same for the same arguments.

Use Python's LRU caching decorator to help improve performance when this function is called multiple times with the same arguments.

Then use `timeit` to test how performance is affected.

##### Solution 3: Without LRU Cache

In [33]:
from timeit import timeit

In [41]:
def cache(fn):
    print('Initializing cache...')
    cache = {}
    def inner(*args):
        key = args
        if key in cache:
            print('cache hit', end='')
            return cache[args]
        else:
            print('cache miss')
            print(f'running {fn.__name__}() function', end='')
            result = fn(*args)
            cache[args] = result
            return result
    return inner

In [50]:
@cache
def add(x, y):
    sleep(2)
    return x + y

Initializing cache...


In [51]:
add(10, 10)

cache miss
running add() function

20

In [52]:
add(10, 10)

cache hit

20

In [53]:
timeit('add(20, 10)', globals=globals(), number=1)

cache miss
running add() function

2.0022169040021254

In [54]:
timeit('add(20, 10)', globals=globals(), number=1)

cache hit

9.720000525703654e-05

##### Solution 3: With LRU Cache

In [56]:
from functools import lru_cache

In [57]:
@lru_cache(maxsize=2)
def add_lru(x, y):
    sleep(2)
    return x + y

In [58]:
add_lru(10, 10)

20

In [59]:
add_lru(10, 10)

20

In [60]:
timeit('add_lru(20, 10)', globals=globals(), number=1)

2.0024004410006455

In [61]:
timeit('add_lru(20, 10)', globals=globals(), number=1)

2.9000002541579306e-06

#### Question 4

This is kind of a "bonus" exercise. It's a follow-up to Question 2.

It's also complicated, so don't worry if you are unable to do this one!

In Question 2, we created a decorator that used a global variable for the precision.

Here, we would rather define a decorator that can take that precision as an argument, i.e. we could do something like this:

```
@normalize(2)
def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0
    
@normalize(4)
def sum_squares(*args):
    return sum(x**2 for x in args)

@normalize(8)
def avg(*args):
    if len(args) == 0:
        return 0
    
    numbers = [Decimal(x) for x in args]
    sum_ = sum(numbers)
    return sum_ / len(numbers)
```

As a hint, remember how we created "partial" functions in a previous exercise?

What we'll want to do here is not write a decorator function directly, but instead write a function that will **create** a decorator function, with the precision captured in the decorator function (which will itself then, be a closure).

Something like this:

```
def normalize(precision):
    def decorator(fn):
        def inner(*args, **kwargs):
            # precision passed to normalize is available here
            return result
        return inner
    return decorator
```

In [66]:
def normalize(precision):
    def decorator(func):
        print(f'{func.__name__}() now decorator...')
        def inner(*args):
            result = func(*args)
            return round(result, precision)
        return inner
    return decorator

In [96]:
@normalize(PRECISION)
def perc_diff(x, y):
    try:
        return (y-x) / x * 100
    except ZeroDivisionError:
        return 0

@normalize(PRECISION)
def sum_squares(*args):
    return sum(x**2 for x in args)

@normalize(PRECISION)
def avg(*args):
    if len(args) == 0:
        return 0
    
    numbers = [Decimal(x) for x in args]
    sum_ = sum(numbers)
    return sum_ / len(numbers)

perc_diff() now decorator...
sum_squares() now decorator...
avg() now decorator...


In [97]:
perc_diff(3, 4)

33.33

In [98]:
sum_squares(2, 5, 3, 7, 9, 10.2355)

272.77

In [99]:
avg(2.8, 2.1, 9.0, 10, 0.6, 0.2353)

Decimal('4.12')